# Intro

In [ ]:
# imports required to run the code.
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import sklearn
import autograd.numpy as np_
import numpy as np
from autograd import grad

In [ ]:
# preparing dataset.

dfo = pd.read_csv('healthcare-dataset-stroke-data.csv')
dfo = dfo.fillna(0)

# ---------------------------------------- #

dfo.head()

In [ ]:
# preparing dataset.

dfo = pd.read_csv('healthcare-dataset-stroke-data.csv')
dfo = dfo.fillna(0)

dfo = dfo.sort_values(by=['bmi'], ascending=True)
dfo = dfo.reset_index(drop = True)

# ---------------------------------------- #

df = pd.get_dummies(dfo) # converte dados não numéricos.
df = df.drop(columns=['gender_Other', 'gender_Male']) # remoção de dados redundantes.
df.rename(columns={"gender_Female": "gender"}, inplace = True)

# ---------------------------------------- #
df.head()

In [ ]:
df['stroke'].value_counts()

# Amostragem

O primeiro passo para garantir uma análise mais próxima da realidade é a remoção de parcialidades, no nosso caso, identificamos que nossos dados eram majoritariamente de pessoas que não sofreram de AVC's, portanto havia um viés tendendo a essa direção o que poderia reduzir a credibilidade de nosso algoritmo.

Para contornar tal problema, decidimos limitar nosso tamanho de amostra para que houvesse uma relação mais próxima de 1 para 1 entre casos em que um individuo sofreu ou não um AVC. 

In [ ]:
# normalizing dataframe.

filtery = (df['stroke'] == 0)
dfo1 = df[filtery]

dfo1 = dfo1.iloc[:300]

filtery = (df['stroke'] == 1)
dfo2 = df[filtery]

dfo = dfo1.append(dfo2)

# ----------------- #

# excluding more redundant data.
dfa = dfo.drop(columns=['stroke','id','ever_married_No','Residence_type_Rural','smoking_status_formerly smoked']).astype(float)

## Separando dados em teste e treino

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dfa, dfo['stroke'], train_size=0.5)
y_train = y_train.replace(to_replace=0,value=-1)
y_test = y_test.replace(to_replace=0,value=-1)
y_train.value_counts()

In [ ]:
# To Numpy
X_train,X_test,y_train,y_test = X_train.to_numpy(),X_test.to_numpy(),y_train.to_numpy(),y_test.to_numpy()

# Métodos e funções

- acurracy

- eqm

In [ ]:
# ---------------------------------------- #

def predict( w, b, pontos ): # Uses data obtained to extrapolate real value.
    est = w.T @ pontos + b
    return est


def accuracy(y_test, y_est): # Compares extrapolated data with real data.
    return np.mean(np.sign(y_test) == np.sign(y_est))

def eqm( parametros ): # 
    w, b, pontos, val = parametros
    est = w.T @ pontos + b
    mse = np_.mean( (est - val)**2)
    return mse

# ---------------------------------------- #

# Teste de acurácia

In [ ]:
# Generating data for accuracy test.
# Current values are going to take a while.

results = []
for _ in range(10):
    g = grad(eqm)

    pontos = X_train.T
    alvos = (y_train).astype(float)

    w = np.random.randn( pontos.shape[0],1)
    b = 0.0
    alpha = 0.000001

    for n in range(10000):
        grad_ = g( (w, b, pontos, alvos) )
        w -= alpha*grad_[0]
        b -= alpha*grad_[1]

    y_pred = (predict( w, b, X_test.T )).reshape((y_test.shape))

    results.append(accuracy(y_test, y_pred))

In [ ]:
# prints accuracy data.

print(np.array(results).mean())
print(np.array(results).std())

Com base nos parametros normalisados, podemos afirmar que nosso classificador tem um funcionamento teórico bom - acertando em media 60% das vezes.

In [ ]:
# Generating prediction values.

g = grad(eqm)

pontos = X_train.T
alvos = (y_train).astype(float)

w = np.random.randn( pontos.shape[0],1)
b = 0.0
alpha = 0.000001
print(w)
for n in range(1000):
    grad_ = g( (w, b, pontos, alvos) )
    w -= alpha*grad_[0]
    b -= alpha*grad_[1]

y_pred = (predict( w, b, X_test.T )).reshape((y_test.shape))

# Visualizando os fatores de relevância

In [ ]:
from collections import OrderedDict

columns = list(dfa.columns)
res = {columns[i]: float(w[i]) for i in range(len(w))}
sorted(res.items(), key=lambda t: t[1], reverse=True)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

print(dfa)
df_features = pd.DataFrame(X_test)
print(df_features)
df_rotulo = pd.DataFrame(y_test)
tree = DecisionTreeClassifier(criterion='entropy')

# Agora, vamos usar o método .fit() para ajustar os parâmetros da árvore:
tree.fit(df_features, df_rotulo)

# Podemos visualizar a árvore de decisão em uma figura!
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure( figsize=(20,20) )
a = plot_tree(tree, feature_names=df_features.columns, fontsize=15, 
              node_ids=False, impurity=False, filled=True)

#a